* 1. PF이름 바탕으로 파일 열기
* 2. 연 파일에서 단어/문장 분리하기
* 3. 단어/문장들을 바탕으로 필요한 정보 추출하기

In [1]:
# -*- coding:utf-8 -*-
#import 및 상수들

import fitz
import os
import pandas as pd
import logging
import unicodedata

ABS_path = ['ABS_data']
ABSTB_path = ['ABSTB_data']
ABCP_path = ['ABCP_data']
company = pd.DataFrame(pd.read_csv('./회사.csv'))['회사'].tolist()
print(company)

신용등급_조사할_회사 = pd.read_csv('./신용등급_조사할_회사.csv')
신용등급_조사할_회사.head()
신용등급_조사할_회사_리스트 = 신용등급_조사할_회사['회사'].tolist()

상위행정구역_딕셔너리 = {
    "서울특별시" : 1,
    "서울시": 1,
    "부산광역시": 1,
    "인천광역시": 1,
    "대구광역시": 1,
    "대전광역시": 1,
    "광주광역시": 1,
    "울산광역시": 1,
    "세종특별자치시": 1,
    '경기도': 1,
    "충청북도": 1,
    "충청남도": 1,
    "전라남도": 1,
    "전라북도": 1,
    "경상북도": 1,
    "경상남도": 1,
    "강원도": 1,
    "강원특별자치도": 1,
    "전북특별자치도": 1,
    "제주특별자치도": 1,
    '수원특례시' : 1
}

# 세부유형 : 일반유형
사업용도_세부종목_딕셔너리 = {
    "도시형생활주택" : "공동주택",
    "도시형 생활주택" : "공동주택",
    "다세대주택" : "공동주택",
    "다세대 주택" : "공동주택",
    "아파트" : "공동주택",
    "재개발" : "공동주택",
    "재건축" : "공동주택",
    "정비사업" : "공동주택",
    "임대주택" : "공동주택",
    "임대 주택" : "공동주택",
    "청년주택" : "공동주택",
    "청년 주택" : "공동주택",
    "주택": "공동주택",

    "도시개발" : "용도복합",
    "주상복합" : "공동주택/상업업무",
    "오피스텔" : "오피스텔",
    "물류" : "산업시설",
    "공장" : "산업시설",
    

    "오피스" : "상업업무",
    "리테일" : "상업업무",
    "자족시설" : "상업업무",
    "상업" : "상업업무",
    "워터파크" : "레저시설",

    "지식산업센터" : "특수용도",
    "생활형숙박시설" : "특수용도",
    "생활형 숙박시설" : "특수용도",
    "생활숙박시설" : "특수용도",
    "생활 숙박시설" : "특수용도",

    "토지개발" : "기타",
    "부지개발" : "기타",
    "인프라": "기타",
    "사회기반시설": "기타",
    "LNG": "기타",
    "에너지": "기타",

    "산업" : "산업시설",
}

['삼성물산주식회사', '현대건설(주)', '(주)대우건설', '현대엔지니어링(주)', '지에스건설(주)', '디엘이앤씨(주)', '주식회사포스코이앤씨', '롯데건설(주)', '에스케이에코플랜트(주)', '(주)호반건설', '에이치디씨현대산업개발(주)', '(주)한화', '디엘건설(주)', '대방건설(주)', '중흥토건(주)', '(주)태영건설', '제일건설(주)', '계룡건설산업(주)', '코오롱글로벌(주)', '(주)서희건설', '금호건설(주)', '동부건설(주)', '아이에스동서(주)', '(주)케이씨씨건설', '우미건설(주)', '(주)반도건설', '한신공영(주)', '쌍용건설(주)', '(주)호반산업', '에이치엘디앤아이한라(주)', '(주)동원개발', '신세계건설(주)', '삼성엔지니어링(주)', '에스지씨이테크건설(주)', '두산건설(주)', '(주)동양건설산업', '(주)에이치제이중공업', '(주)한양', '(주)금강주택', '(주)라인건설', '효성중공업(주)', '두산에너빌리티(주)', '화성산업(주)', '(주)라인산업', '(주)대광건영', '양우건설(주)', '진흥기업(주)', '(주)서한', '(주)금성백조주택', '중흥건설(주)', '씨제이대한통운(주)', '대보건설(주)', '엘티삼보(주)', '(주)시티건설', '자이씨앤에이(주)', '보광종합건설(주)', '(주)우미개발', '(주)케이알산업', '(주)성도이엔지', '신동아건설(주)', '동문건설(주)', '남광토건(주)', '디에스종합건설(주)', '자이에스앤디(주)', '(주)대원', '동원건설산업(주)', '(주)태왕이앤씨', '극동건설(주)', '일성건설(주)', '(주)금성백조건설', '에이스건설(주)', '요진건설산업(주)', '(주)우방', '일신건영(주)', '대우산업개발(주)', '경동건설(주)', '삼부토건(주)', '대상건설(주)', '파인건설(주)', '이수건설(주)', '(주)동양', '경남기업(주)', '대방산업개발(주)', '계성건설(주)', '(주)

/var/folders/q_/bfhzy6314fn6kyhg8lzpxn640000gn/T/ipykernel_87632/1855634780.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def 파일_불러오기(directory_path, partial_filename_to_check):
    for filename in os.listdir(directory_path):
        if partial_filename_to_check in unicodedata.normalize('NFC', filename):
            print(directory_path)
            print(filename)
            pdf_doc = fitz.open(f'{directory_path}/{filename}')
            return pdf_doc
    print('error')

def 주소(단어들):
    for index, 단어 in enumerate(단어들):
        if 상위행정구역_딕셔너리.get(단어):
            return 단어들[index],단어들[index+1],단어들[index+2],단어들[index+3].rstrip(',')
    return 0,0,0,0

def 주소_건물용도_문장_찾기(문장들):
    for 문장 in 문장들:
        for 상위행정구역 in 상위행정구역_딕셔너리.keys():
            if 상위행정구역 in 문장:
                return 문장
    return ''


브릿지론_분류기준 = ['브릿지론', '인허가', '부지를 매입', '건물을 매입']
본PF_분류기준 = ['인허가 완료', '착공']
def 브릿지론_찾기(단어들):
    for 단어 in 단어들:
        for 분류단어 in 브릿지론_분류기준:
            if 분류단어 in 단어:
                return '브릿지론'
        for 분류단어 in 본PF_분류기준:
            if 분류단어 in 단어:
                return '본 PF'
    return '0'


def 사업용도_세부종목_확인(문장):
    for 세부종목 in 사업용도_세부종목_딕셔너리.keys():
        if 세부종목 in 문장:
            사업용도 = 사업용도_세부종목_딕셔너리.get(세부종목)
            if 세부종목 == "재개발" or 세부종목 == "재건축" or 세부종목 == "정비사업":
                세부종목 = "아파트"
            if 세부종목 == "청년주택" or 세부종목 == "청년 주택":
                세부종목 == "임대주택"
            if 세부종목 == "주택" or 세부종목 == "상업" or 세부종목 == "산업":
                세부종목 = "-"
            return 사업용도,세부종목
    return 0,0

def 사업용도_세부종목_확인_2(단어들):
    주사업용도_변수 = {"공동주택" : False,"오피스텔" : False,"산업시설" : False,
                "상업업무" : False,"레저시설" : False,"용도복합" : False,
                 "특수용도" : False,"기타" : False,"태영건설" : False}
    
    검색할_단어_딕셔너리 = {
        #공동주택
        "아파트" : False,
        "주거복합" : False,
        '재개발' : False, 
        '재건축' : False, 
        '도시형생활주택' : False,
        '다세대' : False,
        '노인주택' : False,
        '임대주택' : False,
        '청년주택' : False,
        '공동주택' : False, # 예외처리
        '주상복합' : False, # 예외처리

        #오피스텔
        '오피스텔'  : False,

        #산업시설
        '물류' : False,
        '냉동' : False,
        '냉장' : False,
        '공장' : False,

        #상업업무
        '오피스' : False,
        '업무시설' : False,
        '리테일' : False,
        '판매시설': False,
        '근린생활시설': False,
        '호텔': False,
        '쇼핑몰': False,
        '복합쇼핑몰': False,
        '복합환승센터': False,
        '상업': False,      # 예외처리
        '자족시설': False,   # 예외처리

        #레저시설
        '워터파크': False,
        '테마파크': False,

        #용도복합
        '도시개발': False,

        #특수용도
        '지식산업센터': False,
        '생활형숙박시설': False,

        #기타
        '토지개발': False,
        '인프라': False,
        '사회기반시설': False,
        '전철': False,
        'LNG': False,
        '에너지': False,

        #태영건설
        '태영건설' : False
    }
    
    for 단어 in 단어들:
        for 비교_단어, 존재여부 in 검색할_단어_딕셔너리.items():
            if 비교_단어 in 단어:
                검색할_단어_딕셔너리[비교_단어] = True
            
            elif 단어.startswith('노인') and 단어.endswith('주택'):
                검색할_단어_딕셔너리['노인주택'] = True
    if 검색할_단어_딕셔너리['태영건설']:
        return 'Default', ['Default']


    cnt = 0
    검색된_단어 = []
    for key, value in 검색할_단어_딕셔너리.items():
        if value == True:
            if key == '주거복합' or key == '재개발' or key == '재건축':
                검색된_단어.append('아파트')
            elif key == '청년주택':
                검색된_단어.append('임대주택')
            elif key == '냉동' or key == '냉장':
                검색된_단어.append('물류')
            elif key == '업무시설':
                검색된_단어.append('오피스')
            elif key == '판매시설' or key == '근린생활시설':
                검색된_단어.append('리테일')
            elif key == '복합쇼핑몰' or key == '복합환승센터':
                검색된_단어.append('쇼핑몰')
            elif key == '사회기반시설' or key == '전철':
                검색된_단어.append('인프라')
            else:
                검색된_단어.append(key)
    검색된_단어 = list(set(검색된_단어))
    

    # 주사업용도 결정하기 -> 검색된 단어를 바탕으로...!
    if 검색할_단어_딕셔너리['아파트'] or 검색할_단어_딕셔너리['주거복합'] or 검색할_단어_딕셔너리['재개발'] or 검색할_단어_딕셔너리['재건축']\
        or 검색할_단어_딕셔너리['도시형생활주택'] or 검색할_단어_딕셔너리['다세대'] or 검색할_단어_딕셔너리['노인주택']\
        or 검색할_단어_딕셔너리['임대주택'] or 검색할_단어_딕셔너리['청년주택'] or 검색할_단어_딕셔너리['주상복합'] or 검색할_단어_딕셔너리['공동주택']:
        주사업용도_변수["공동주택"] = True

    if 검색할_단어_딕셔너리['오피스텔']:
        주사업용도_변수['오피스텔'] = True

    if 검색할_단어_딕셔너리['물류'] or 검색할_단어_딕셔너리['냉동'] or 검색할_단어_딕셔너리['냉장'] or 검색할_단어_딕셔너리['공장']:
        주사업용도_변수['산업시설'] = True

    if 검색할_단어_딕셔너리['오피스'] or 검색할_단어_딕셔너리['업무시설'] or 검색할_단어_딕셔너리['리테일'] or 검색할_단어_딕셔너리['판매시설']\
        or 검색할_단어_딕셔너리['근린생활시설'] or 검색할_단어_딕셔너리['호텔'] or 검색할_단어_딕셔너리['쇼핑몰'] or 검색할_단어_딕셔너리['복합쇼핑몰']\
            or 검색할_단어_딕셔너리['복합환승센터'] or 검색할_단어_딕셔너리['상업'] or 검색할_단어_딕셔너리['자족시설']:
        주사업용도_변수['상업업무'] = True

    if 검색할_단어_딕셔너리['워터파크'] or 검색할_단어_딕셔너리['테마파크']:
        주사업용도_변수['레저시설'] = True

    if 검색할_단어_딕셔너리['도시개발']:
        주사업용도_변수['용도복합'] = True

    if 검색할_단어_딕셔너리['지식산업센터'] or 검색할_단어_딕셔너리['생활형숙박시설']:
        주사업용도_변수['특수용도'] = True

    if 검색할_단어_딕셔너리['토지개발'] or 검색할_단어_딕셔너리['인프라'] or 검색할_단어_딕셔너리['사회기반시설']\
        or 검색할_단어_딕셔너리['전철'] or 검색할_단어_딕셔너리['LNG'] or 검색할_단어_딕셔너리['에너지']:
        주사업용도_변수['기타'] = True

    # 공동주택 만, 상업, 자족시설, 주상복합
    사업용도_세부종목 = []
    주사업용도 = ""
    if len(검색된_단어) == 1:
        if 검색된_단어[0] == '공동주택':
            사업용도_세부종목.append('아파트')
            주사업용도 = '공동주택'
            print('공동주택만')
        elif 검색된_단어[0] == '상업':
            주사업용도 = '상업업무'
            사업용도_세부종목.append('오피스')
            사업용도_세부종목.append('리테일')
            print('상업')
        elif 검색된_단어[0] == '자족시설':
            주사업용도 = '상업업무'
            사업용도_세부종목.append('오피스')
            사업용도_세부종목.append('리테일')
            print('자족시설')
        elif 검색된_단어[0] == '주상복합':
            print('주상복합')
            주사업용도 = '공동주택'
            사업용도_세부종목.append('아파트')
            사업용도_세부종목.append('근린생활시설')
        elif 검색된_단어[0] == '도시개발':
            주사업용도 = '용도복합'
            사업용도_세부종목.append('도시개발')
            print('도시개발')
        else:
            사업용도_세부종목=[검색된_단어[0]]
            ##
            if 주사업용도_변수['공동주택']:
                주사업용도 = '공동주택'
                사업용도_세부종목 = 검색된_단어
            elif 주사업용도_변수['오피스텔']:
                주사업용도 = '오피스텔'
                사업용도_세부종목 = 검색된_단어
            elif 주사업용도_변수['산업시설'] or 주사업용도_변수['레저시설'] or 주사업용도_변수['특수용도'] or 주사업용도_변수['기타']:
                주사업용도 = ''
                if 주사업용도_변수['산업시설']:
                    if 주사업용도 != '':
                        주사업용도 += '.'
                    주사업용도 += '산업시설'
                if 주사업용도_변수['레저시설']:
                    if 주사업용도 != '':
                        주사업용도 += '.'
                    주사업용도 += '레저시설'
                if 주사업용도_변수['특수용도']:
                    if 주사업용도 != '':
                        주사업용도 += '.'
                    주사업용도 += '특수용도'
                if 주사업용도_변수['기타']:
                    if 주사업용도 != '':
                        주사업용도 += '.'
                    주사업용도 += '기타'
                사업용도_세부종목 = 검색된_단어
            elif 주사업용도_변수['상업업무']:
                주사업용도 = '상업업무'
                사업용도_세부종목 = 검색된_단어
            
                
    #여러개인 경우
    else:
        # 주사업 용도를 기반으로 또다시 정리
        if '상업' in 검색된_단어:
            검색된_단어.remove('상업')

        if '자족시설' in 검색된_단어:
            검색된_단어.remove('자족시설')

        if 주사업용도_변수['공동주택']:
            주사업용도 = '공동주택'
            사업용도_세부종목 = 검색된_단어
        elif 주사업용도_변수['오피스텔']:
            주사업용도 = '오피스텔'
            사업용도_세부종목 = 검색된_단어
        elif 주사업용도_변수['산업시설'] or 주사업용도_변수['레저시설'] or 주사업용도_변수['특수용도'] or 주사업용도_변수['기타']:
            주사업용도 = ''
            if 주사업용도_변수['산업시설']:
                if 주사업용도 != '':
                    주사업용도 += '.'
                주사업용도 += '산업시설'
            if 주사업용도_변수['레저시설']:
                if 주사업용도 != '':
                    주사업용도 += '.'
                주사업용도 += '레저시설'
            if 주사업용도_변수['특수용도']:
                if 주사업용도 != '':
                    주사업용도 += '.'
                주사업용도 += '특수용도'
            if 주사업용도_변수['기타']:
                if 주사업용도 != '':
                    주사업용도 += '.'
                주사업용도 += '기타'
            사업용도_세부종목 = 검색된_단어
        elif 주사업용도_변수['상업업무']:
            주사업용도 = '상업업무'
            사업용도_세부종목 = 검색된_단어
    
    return 주사업용도, 사업용도_세부종목
        
print(사업용도_세부종목_확인_2("추진되는 공동주택, 오피스텔 및 판매시설 개발사업, 그리고 여타 지역에서의 부동산 운영사업 또".split(' ')))
    
print(사업용도_세부종목_확인("차주는 인천광역시 연수구 송도동 30-5번지 내 B3BL 일원에서 주상복합건물 등을 신축하여 분양하는 사업(이하 ‘본 사업’)을 관리형토지신탁(수탁자: 케이비부동산신탁㈜) 방식으로 추진 중인 사업주이다"))

('공동주택', ['리테일', '공동주택', '오피스텔', '오피스'])
('공동주택/상업업무', '주상복합')


In [3]:
import re
######################
# 신용 보간 기관 관련 함수 #
######################


# 시작 ~ 끝 단어를 통한 문장 추출
def extract_string_between(input_string, start_string, end_string):
    start_index = input_string.find(start_string)
    if start_index == -1:
        return None  # 시작 문자열이 발견되지 않으면 None을 반환
    
    end_index = input_string.find(end_string, start_index + len(start_string))
    if end_index == -1:
        return None  # 끝 문자열이 발견되지 않으면 None을 반환
    
    return (input_string[start_index + len(start_string):end_index+2], input_string[end_index:])

def 시공능력평가표상(기관명, company):
    for i in company:
        if 기관명 in i or i in 기관명:
            return True
    return False

def 신용보강기관_명_기관종류_형태(신용보강문장_리스트):
    신용보강기관_명 = ''
    신용보강기관_종류 = ''
    신용보강기관_형태 = ''

    한줄_신용보관기관명 = []
    한줄_신용보관기관종류 = []
    한줄_신용보관기관형태 = []
    for text in 신용보강문장_리스트:
        text = text.replace('㈜','').replace('(주)','')
        text = text.replace('(','').replace(')','')
        #의
        entities = []
        entities_종류 = []
        entities_보강형태 = []
        
        if '신한' in text:
            print('debug')

        pattern = r'\b(\w+공사)'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(1))

        # pattern = r'\b(\w+증권)'
        # match = re.search(pattern, text)
        # if match:
        #     entities.append(match.group(1))

        pattern = r'\b(\w+은행)'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(1))

        pattern = r'\b(\w+기관)'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(1))

        
        # pattern = r'\b(\w+)시'
        # match = re.search(pattern, text)
        # if match:
        #     entities.append(match.group(1))

        
        # pattern = r'\b(\w+)군'
        # match = re.search(pattern, text)
        # if match:
        #     entities.append(match.group(1))

        
        # pattern = r'\b(\w+)도'
        # match = re.search(pattern, text)
        # if match:
        #     entities.append(match.group(1))


        pattern = r'\b(\w+)신탁'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(1))

        pattern = r'\b(\w+)의\s(\w+)\s대한\s(\w+)의'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(3))
        else:
            pattern = r'\b(\w+)의\b'
            match = re.search(pattern, text)
            if match:
                entities.append(match.group(1))

        #A 및 B와(과) 체결한
        pattern = r'\b(\w+)\s및\s(\w+)(와|과)\s체결한\b'
        match = re.search(pattern, text)
        if match:
            entities.append(match.group(1))
            entities.append(match.group(2))
        else:
            # B와(과) 체결한
            pattern = r'\b(\w+)(와|과)\s체결한\b'
            match = re.search(pattern, text)
            if match:
                entities.append(match.group(1))
        
        for i in entities:
            i.replace('(주)', '').replace('㈜','')
        
        entities = list(set(entities))
        
        for 기관명 in entities:
            if '증권' in 기관명:
                entities_종류.append('증권사')
            elif '은행' in 기관명:
                entities_종류.append('은행')
            elif '기관' in 기관명 or '공사' in 기관명 or 기관명.endswith('시') or 기관명.endswith('군') or 기관명.endswith('도'):
                entities_종류.append('공공')
            elif '신탁' in 기관명:
                entities_종류.append('신탁회사')
            elif '건설' in 기관명 or 시공능력평가표상(기관명, company):
                entities_종류.append('시공사')
            else:
                entities_종류.append('기타')
            # ~증권 -> 증권사
            # ~은행 -> 은행
            # ~기관, ~공사 -> 공공
            # 시공능력평가표상 존재 -> 시공사
            # 나머지 -> 기타
                
        # 신용보강형태 뽑기
        pattern = r'채권\s*매입'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('채권매입')
        else:
            pattern = r'(매입)'
            match = re.search(pattern, text)
            if match:
                entities_보강형태.append(match.group(1))

        pattern = r'채무\s*인수'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('채무인수')
        else:
            pattern = r'(인수)'
            match = re.search(pattern, text)
            if match:
                entities_보강형태.append(match.group(1))

        pattern = r'연대\s*보증'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('연대보증')
        
        pattern = r'지급\s*보증'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('지급보증')

        pattern = r'자금\s*보충'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('자금보충')

        pattern = r'책임\s*준공'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('책임준공')

        pattern = r'책임\s*분양'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append('책임분양')

        pattern = r'(이자율)'
        match = re.search(pattern, text)
        if match:
            entities_보강형태.append(match.group(1))

        if not entities:
            entities.append('-')
        if not entities_종류:
            entities_종류.append('-')
        if not entities_보강형태:
            entities_보강형태.append('-')
        한줄_신용보관기관명.append('/'.join(entities))
        한줄_신용보관기관종류.append('/'.join(entities_종류))
        한줄_신용보관기관형태.append('/'.join(entities_보강형태))

    신용보강기관_명 += '.'.join(한줄_신용보관기관명)
    신용보강기관_종류 += '.'.join(한줄_신용보관기관종류)
    신용보강기관_형태 += '.'.join(한줄_신용보관기관형태)
    return 신용보강기관_명, 신용보강기관_종류, 신용보강기관_형태

매입확약여부_확인_리스트 = [r'인수\s*의무',r'인수\s*확약',r'매입\s*확약',r'매입\s*의무',r'신용\s*공여']
매입보장여부_확인_리스트 = [r'매입\s*보장',r'매입\s*약정',r'인수\s*약정']

def 매입보장_매입확약(모든_글자들):
    _매입확약여부 = False
    _매입보장여부 = False
    
    for pattern in 매입확약여부_확인_리스트:
        match = re.search(pattern, 모든_글자들)
        if match:
           _매입확약여부 = True

    if not _매입확약여부:
        for pattern in 매입보장여부_확인_리스트:
            match = re.search(pattern, 모든_글자들)
            if match:
                _매입확약여부 = False
                _매입보장여부 = True

    pattern = r'신용\s*공여\s*어음'
    match = re.search(pattern, 모든_글자들)
    if match:
        _매입확약여부 = True
        _매입보장여부 = False
        
    if _매입확약여부 == True and _매입보장여부 == False:
        return '매입확약'
    elif _매입확약여부 == False and _매입보장여부 == True:
        return '매입보장'
    elif _매입확약여부 == False and _매입보장여부 == False:
        return '-'
    
def 대주_대주단_문단_찾기(문단들):
    for text in 문단들:
        if '대주' in text:
            return text
    return ''

# pattern = r'(\d+조\s*(\d+(?:,\d+)*))억\w*\b'
def 대주단_문단에서_가장큰_금액_찾기(문단):
    result = '-'
    final_result = []
    pattern = r'(\d+)조\s*(\d+(?:,\d+)*)억\w*\b'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = int(match[i][0])*10000 + int(match[i][1].replace(',',''))
        result = max(match)
        final_result.append(result)

    pattern = r'(\d+)조원'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = match[i]
        result = max(list(map(int, match)))
        final_result.append(result*10000)
    
    # pattern = r'\b(\d+(?:,\d+)*)억\b'
    pattern = r'\b(\d+(?:,\d+)*)억\w*\b'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = match[i].replace(',','')
        result = max(list(map(int, match)))
        final_result.append(result)
        
    if not final_result:
        return '-'
    return(str(max(final_result)))

print(대주단_문단에서_가장큰_금액_찾기(' 원금 합계 1,150억원의 한도의 대출채권 및'))

def 분양률_찾기(문장):
    result = ''
    pattern = r'분양률\w*\s+\w*(\d+)(\.\d+)?%'
    matches = re.search(pattern, 문장)
    if matches:
        temp_text = matches.group()

        pattern = r'(\d+)(\.\d+)?'
        matches = re.search(pattern, temp_text)
        if matches:
            result = matches.group()
            return result
    return "-"

print(분양률_찾기('5월말 기준 준공완료 상태로 분양률은 63.8%이다. \n차주는 본 사업과 관련하여 2016년 1월 트리니티진주(유)(Tr.B 대주)를 포함한 대주들과 총 1,700'))

1150
63.8


In [4]:
# import re
# text = '학익4주택재개발정비사업조합의 대출채무대출원금에 대한 주택도시보증공사의 PF대출보증'
# entities = []

# pattern = r'\b(\w+)의\s(\w+)\s대한\s(\w+)의'
# match = re.search(pattern, text)
# print(match)
# if match:
#     entities.append(match.group(3))
# else:
#     pattern = r'\b(\w+)의\b'
#     match = re.search(pattern, text)
#     print(match)
#     if match:
#         entities.append(match.group(1))

# #A 및 B와(과) 체결한
# pattern = r'\b(\w+)\s및\s(\w+)(와|과)\s체결한\b'
# match = re.search(pattern, text)
# print(match)
# if match:
#     entities.append(match.group(1))
#     entities.append(match.group(2))
# else:
#     # B와(과) 체결한
#     pattern = r'\b(\w+)(와|과)\s체결한\b'
#     match = re.search(pattern, text)
#     print(match)
#     if match:
#         entities.append(match.group(1))
        






In [5]:
from collections import deque

경로_딕셔너리 = {
   './ABS_data.csv' : "file/ABS",
    './ABSTB_data.csv' : "file/ABSTB",
    './ABCP_data.csv' : "file/ABCP",
}

def 엑셀파일읽기(path):
    logging.basicConfig(filename=f"log/{경로_딕셔너리.get(path)[5:]}_logging.csv", level=logging.INFO, format="%(message)s", encoding='utf-8')
    logging.info(f'발행인,시도,시구,동읍,상세주소,사업용도,사업용도세부종목,브릿지론/본PF,신용보강기관명,신용보강기관종류,신용보강형태,매입보장/확약,대출약정규모,분양률,문장')
    data = pd.read_csv(path)
    dropped_data = pd.DataFrame(data)
    for i in data.columns.to_list():
        if i not in ['발행인', '한국신용평가', '한국기업평가', 'NICE신용평가', '서울신용평가']:
            dropped_data = dropped_data.drop(i, axis=1)
    print(dropped_data)

    중복체크 = dict()
    for index, row in dropped_data.iterrows():

        시도 = ''
        시구 = ''
        동읍 = '' 
        상세주소 = ''
        브릿지론본PF  = ''
        신용보강기관종류2  = ''
        신용보강기관명2 = ''
        신용보강형태2 = ''
        사업용도 = ''
        사업용도세부종목 = ''
        트렌치 = ''
        트레치수 = ''
        대출약정규모 = ''
        분양률 = ''
        매입보장_확약 = ''

        신용보강기관문장 = []
        발행인, 한국신용평가, 한국기업평가, NICE신용평가, 서울신용평가 = row[:]
        if 중복체크.get(발행인):
            logging.info(중복체크.get(발행인))
            continue
    
        
        평가기관상태 = { '한국신용평가' : 한국신용평가,
                  '한국기업평가' : 한국기업평가,
                  'NICE신용평가' : NICE신용평가,
                  '서울신용평가' : 서울신용평가
                  }
        
        text_list_dict = { '한국신용평가' : ("⚫", "\n"),
                  '한국기업평가' : ("", "\n"),
                  'NICE신용평가' : ("", "\n"),
                  '서울신용평가' : ("", "\n")
                  }
        if pd.isna(한국신용평가) and pd.isna(한국기업평가) and pd.isna(NICE신용평가) and pd.isna(서울신용평가):
            logging.info(f'{발행인},자료없음')
            continue
        for key, value in 평가기관상태.items():
            if not pd.isna(value):
                경로 = f'./{경로_딕셔너리.get(path)}/{key}'
                print(경로)
                pdf = 파일_불러오기(경로, 발행인)
                if not pdf:
                    logging.info(f'{발행인},파일 열기 오류')
                    continue
                단어들 = []
                문장들 = []
                모든_글자들 = ''
                문단들 = ''
                for i in range(pdf.page_count - 1):
                    page = pdf.load_page(i)
                    단어들.extend(page.get_text().replace('\n', '').split())
                    문장들.extend(page.get_text().replace('\n', '').split('.'))
                    문단들 += page.get_text()
                    모든_글자들 += page.get_text()

                문단들 = 문단들.split('\n\n')
                    
                #################################
                # 분양률
                #################################
                if not 분양률 or 분양률 == '-':
                    분양률 = 분양률_찾기(모든_글자들)

                #################################
                # 전체대출약정규모
                #################################
                if not 대출약정규모 or 대출약정규모 == '-':
                    print(발행인)
                    대출약정규모 = 대주단_문단에서_가장큰_금액_찾기(대주_대주단_문단_찾기(문단들))


                #################################
                # 매입보장 / 매입확약
                #################################
                if not 매입보장_확약 or 매입보장_확약 == '-':
                    매입보장_확약 = 매입보장_매입확약(모든_글자들)

                #################################
                # 신용보강기관명 / 신용보강형태
                #################################
                # result_list 가 하나의 문장!
                if not 신용보강기관문장:
                    status = True
                    result_list = []
                    #한국기업평가이면
                    if key == '한국기업평가':
                        temp_result = extract_string_between(모든_글자들, '주요평정요인','■')
                        if temp_result:
                            모든_글자들 = temp_result[0]
                            모든_글자들 += '■'
                            while True:
                                if status:
                                    temp_result = extract_string_between(모든_글자들, '', '')
                                    if temp_result:
                                        result_list.append(temp_result[0].replace('\n','').replace(',',''))
                                        모든_글자들 = temp_result[1]
                                    else:
                                        temp_result = extract_string_between(모든_글자들, '', '■')
                                        if temp_result:
                                            _temp_result = temp_result[0]
                                            idx = _temp_result.find('Structured Finance')
                                            if idx:
                                                _temp_result = _temp_result[:idx]
                                            
                                            idx = _temp_result.find('www.korearatings.com')
                                            if idx:
                                                _temp_result = _temp_result[:idx]

                                            idx = _temp_result.find('본평가평가일')
                                            if idx:
                                                _temp_result = _temp_result[:idx]
                                            
                                            result_list.append(_temp_result.replace('\n','').replace(',',''))
                                        status = False
                                elif not status:
                                    break

                    

                    while True:
                        if key == '한국기업평가':
                            break
                        if status:
                            temp_result = extract_string_between(모든_글자들, text_list_dict.get(key)[0], text_list_dict.get(key)[1])
                            if temp_result:
                                result_list.append(temp_result[0].replace('\n','').replace(',',''))
                                모든_글자들 = temp_result[1]
                            else:
                                status = False
                            # 아래식 수행
                        elif not status:
                            break
                    신용보강기관문장 = result_list[:]
                
                #신용 기관명이 없으면 채우기!!
                if not 신용보강기관명2:
                    신용보강기관명2, 신용보강기관종류2, 신용보강형태2 = 신용보강기관_명_기관종류_형태(result_list)
                    
                


                ############################################################################
                # 시도, 시구, 동읍, 상세주소, 사업용도, 사업용도 세부종목
                ############################################################################
                _시도, _시구, _동읍, _상세주소 = 주소(단어들)
                _사업용도, _사업용도세부종목_리스트 = 사업용도_세부종목_확인_2(단어들)
                _사업용도세부종목 = '.'.join(_사업용도세부종목_리스트)

                if _사업용도 == 0 and _사업용도세부종목 == 0 and not 사업용도 and not 사업용도세부종목:
                    _사업용도, _사업용도세부종목 = 사업용도_세부종목_확인(단어들)
                    if _사업용도 and _사업용도세부종목:
                        _사업용도 = f'{_사업용도}(확인필요)'
                        _사업용도세부종목 = f'{_사업용도세부종목}(확인필요)'

                if not 시도:
                    시도 = _시도
                if not 시구:
                    시구 = _시구
                if not 동읍:
                    동읍 = _동읍
                if not 상세주소:
                    상세주소 = _상세주소
                if not 사업용도:
                    사업용도 = _사업용도
                if not 사업용도세부종목:
                    사업용도세부종목 = _사업용도세부종목

                ############################################################################
                # 브릿지론, 본 PF여부
                ############################################################################
                _브릿지론본PF = 브릿지론_찾기(단어들)
                if not 브릿지론본PF:
                    브릿지론본PF = _브릿지론본PF

        #,{'.'.join(result_list)}
        중복체크[발행인] = f'{발행인},{시도},{시구},{동읍},{상세주소},{사업용도},{사업용도세부종목},{브릿지론본PF},{신용보강기관명2},{신용보강기관종류2},{신용보강형태2},{매입보장_확약},{대출약정규모},{분양률},{'.'.join(result_list).replace(',','')}'
        logging.info(f'{발행인},{시도},{시구},{동읍},{상세주소},{사업용도},{사업용도세부종목},{브릿지론본PF},{신용보강기관명2},{신용보강기관종류2},{신용보강형태2},{매입보장_확약},{대출약정규모},{분양률},{'.'.join(result_list).replace(',','')}')

In [6]:
# ABS_data, ABSTB_data, ABCP_data 3의 파일에 대하여 수행해야함
#  './ABS_data.csv'
#  './ABSTB_data.csv'
#  './ABCP_data.csv'
path = './ABS_data.csv'
엑셀파일읽기(path)


             발행인 한국신용평가 한국기업평가 NICE신용평가 서울신용평가
0     뉴스타제이에이치센터    NaN    NaN      NaN    NaN
1      디비하이엔드제일차    NaN    NaN      NaN    NaN
2        리미티리제삼차    NaN    NaN      NaN    NaN
3      아디아북천안제일차    NaN    NaN      NaN    NaN
4       에이블도산제일차    NaN     A1       A1    NaN
...          ...    ...    ...      ...    ...
1324    케이지야음제일차    NaN    A3+      A3+    NaN
1325    케이지야음제일차    NaN    A3+      A3+    NaN
1326    케이지야음제일차    NaN    A3+      A3+    NaN
1327    케이지야음제일차    NaN    A3+      A3+    NaN
1328         NaN    NaN    NaN      NaN    NaN

[1329 rows x 5 columns]
./file/ABSTB/한국기업평가
./file/ABSTB/한국기업평가
에이블도산제일차(주)_정기(2023-08-07).pdf
에이블도산제일차
상업
./file/ABSTB/NICE신용평가
./file/ABSTB/NICE신용평가
에이블도산제일차(주)_유동화증권_정기_20230607_요지 PDF.pdf
./file/ABSTB/한국기업평가
./file/ABSTB/한국기업평가
고은안골제일차(주)_정기(2023-08-18).pdf
고은안골제일차
상업
./file/ABSTB/NICE신용평가
./file/ABSTB/NICE신용평가
고은안골제일차(주)_유동화증권_정기_20230713_요지 PDF.pdf
./file/ABSTB/NICE신용평가
./file/ABSTB/NICE신용평가
기은센동대문제삼차(주)_유동화증권_본_20240112_요지 P

In [7]:
def extract_string_between(input_string, start_string, end_string):
    start_index = input_string.find(start_string)
    if start_index == -1:
        return None  # 시작 문자열이 발견되지 않으면 None을 반환
    
    end_index = input_string.find(end_string, start_index + len(start_string))
    if end_index == -1:
        return None  # 끝 문자열이 발견되지 않으면 None을 반환
    
    return (input_string[start_index + len(start_string):end_index], input_string[end_index:])

# 예시 문자열
example_string = "이 문자열에서 <start>특정 문자열</start>을 추출해보세요."

# "<start>"와 "</start>" 사이의 문자열 추출
result = extract_string_between(example_string, "<start>", "</start>")
print(result)

서울신용평가_pdf = 파일_불러오기('./file/ABCP/서울신용평가', '위드지엠제이십이차')
한국기업평가_pdf = 파일_불러오기('./file/ABCP/한국기업평가', '케이비프라이드제이차')
한국신용평가_pdf = 파일_불러오기('./file/ABCP/한국신용평가', '스마트오산양산제일차(주)_rs20230921-10')
NICE신용평가_pdf = 파일_불러오기('./file/ABCP/NICE신용평가', '감마스토리제일차')
# print(서울신용평가_pdf,한국기업평가_pdf,한국신용평가_pdf,NICE신용평가_pdf)

pdf_list = [서울신용평가_pdf, 한국기업평가_pdf, 한국신용평가_pdf, NICE신용평가_pdf]
text_list = [("", "\n"), ("", "\n"), ("⚫", "\n"), ("", "\n")]
text_list = [("평가의견", "유동화 개요"), ("주요평정요인", "유동화 개요"), ("한국신용평가(주)", "Structured Finance Rating Report"), ("평가근거", "유동화구조")]
for idx, (pdf, start_end) in enumerate(zip(pdf_list, text_list)):

    print(pdf, start_end)
    text = ''
    for i in range(len(pdf) - 1):
        page = pdf.load_page(i)
        text += page.get_text()
    print('TAG' , idx)
    print(text)

    


('특정 문자열', '</start>을 추출해보세요.')
./file/ABCP/서울신용평가
위드지엠제이십이차.pdf
./file/ABCP/한국기업평가
케이비프라이드제이차(유)_정기(2023-08-07).pdf
./file/ABCP/한국신용평가
스마트오산양산제일차(주)_rs20230921-10.pdf
./file/ABCP/NICE신용평가
감마스토리제일차(주)_유동화증권_본_20230622_요지 PDF.pdf
Document('./file/ABCP/서울신용평가/위드지엠제이십이차.pdf') ('평가의견', '유동화 개요')
TAG 0
Structured Finance 
 
 
CREDIT OPINION 
 
 
www.scri.co.kr ｜ 1 
 
위드지엠제이십이차㈜ 
본평가 
평가일: 2023.09.15 
 
유동화기업어음 신용등급 
구분 
발행금액 
발행(예정)일 
만기(예정)일 
평가등급 
제1회차 ABCP 
20억원 
2023.09.15 
2024.03.15 
A3(sf) 
  
Analyst 
이창욱 전문연구원 
02) 6966-2468 
changwooklee@scri.co.kr 
김상호 전문연구원 
02) 6966-2466 
ksh1028@scri.co.kr 
박인천 실장 
02) 6966-2473 
iseapark@scri.co.kr 
  
유동화 개요 
유동화자산 
대출채권 및 부수권리 (거래유형: PF 대출채권 유동화) 
차주 
동대문어반피에프브이㈜ 
대출약정금 
20억원 (후순위 대출) 
대출만기일 
[기존 만기일] 2023.09.14 
[금번 연장예정 만기일] 2024.03.14 
상환방식 
[원금] 만기일시상환 
 
[이자] 선취 
업무수탁자 
메리츠증권㈜ 
법무법인 
법무법인 리움 
자산관리자 
교보증권㈜ 
주관회사 
교보증권㈜ 
주) 정보제공자: 주관회사 
  
평가의견 
 
서울신용평가㈜는 위드지엠제이십이차㈜(이하 ‘SPC’)가 발행할 예정인 제1회차 유동화기업어음
(이

In [8]:
# 모든_글자들 = ''
# for i in range(한국기업평가_pdf.page_count - 1):
#     page = 한국기업평가_pdf.load_page(i)
#     모든_글자들 += page.get_text()

# status = True
# result_list = []

# temp_result = extract_string_between(모든_글자들, '주요평정요인','■')
# if temp_result:
#     모든_글자들 = temp_result[0]
#     모든_글자들 += '■'
#     print(모든_글자들)
#     while True:
#         if status:
#             temp_result = extract_string_between(모든_글자들, '', '')
#             if temp_result:
#                 result_list.append(temp_result[0].replace('\n',''))
#                 모든_글자들 = temp_result[1]
#             else:
#                 temp_result = extract_string_between(모든_글자들, '', '■')
#                 if temp_result:
#                     result_list.append(temp_result[0])
#                 status = False
#         elif not status:
#             break

# print(result_list)
# entities = []
# pattern = r'\b(\w+)의\b'
# text = ' 지에스건설의 연대보증 의무 '
# text2 = text.replace('㈜','')
# print(text)
# print(text2)
# match = re.search(pattern, text)
# print(match)
# if match:
#     entities.append(match.group(1))
#     print(entities)

In [9]:
import re

# pattern = r'\b(\d+(?:,\d+)*)억\w*\b'

text = '(이하 에게 대출을위해설립된이다.차주와체결한사업및따라 SPC는 제1회   40억원)를   조달한 차주에게 대출(원 금 40억원)을 실행하였고, 이를 기초로       중이다.차주는경상남도창원시진해구안골동산59번지추진중인창원( 이하본사업)의,본사업의시공은.본사업과차 주는 SPC를 비롯한       780억원 한도(Tr.A 490억원 한도, Tr.B 120억원, Tr.C 170억원)고은안골제일차(주)유동화증권을발행하여자금으로실행하기특별목적회사(SpecialPurposeCompany)대출약정에ABSTB(발행금액발행하여금원으로, 상환 및 50개월이     날까지 본 사업에 대한의무를Tr.A, Tr차주에 대한 40억원의 Tr.B 기,한하여.대출채권의이자기간은매3개월(최종 2개월),매초일에.선급된이자는기한의이익상실,조기상환 등을   어떠한     않는다.1회~제18회ABSTB로내에서.다만, 대한기한의이익이경우등에는.유동화제 반 비용은 차주가 부담하며,   대한   인한 유동성 부족은 차주의 후순위 대여로 충당된다.유동화증권 부족등의사유가 경우, SPC로부터 매입하여야 한다. 다만   SPC에게     SPC가     인수할 수 있다.'

# pattern = r'(\d+)(?:,\d+)*억'
# pattern = r'\b(\d+(?:,\d+)*)억\w*\b'
# pattern = r'(\d+)조\b'
pattern = r'(\d+)조\s*(\d+(?:,\d+)*)억\w*\b'
match = re.findall(pattern, text)
if match:
    print(match)


def 대주단_문단에서_가장큰_금액_찾기(문단):
    result = '-'
    final_result = []
    pattern = r'(\d+)조\s*(\d+(?:,\d+)*)억\w*\b'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = int(match[i][0])*10000 + int(match[i][1].replace(',',''))
        result = max(match)
        final_result.append(result)

    pattern = r'(\d+)조원'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = match[i]
        result = max(list(map(int, match)))
        final_result.append(result*10000)
    
    # pattern = r'\b(\d+(?:,\d+)*)억\b'
    pattern = r'\b(\d+(?:,\d+)*)억\w*\b'
    match = re.findall(pattern, 문단)
    if match:
        for i in range(len(match)):
            match[i] = match[i].replace(',','')
        result = max(list(map(int, match)))
        final_result.append(result)
        
    if not final_result:
        return '-'
    return(str(max(final_result)))

대주단_문단에서_가장큰_금액_찾기(text)



'780'